In [ ]:
from astropy.io import fits
import clevar
from clevar.match_metrics.recovery import ClCatalogFuncs as r_cf
from clevar.match_metrics.distances import ClCatalogFuncs as d_cf
from clevar.match_metrics.scaling import ClCatalogFuncs as s_cf
import clevar.match_metrics.scaling as scaling

import pandas as pd
from astropy.table import Table
import astropy
from matplotlib import pyplot as plt
import numpy as np
from numcosmo_py import Ncm, Nc
from numcosmo_py.external.pyssc import pyssc as PySSC
Ncm.cfg_init()

In [ ]:
hdul_ame = fits.open("richness_mcut_SNR 12_ame.fits")
hdul_rfae = fits.open("richness_mcut_SNR_12_Rfae.fits")

lnRmin  = 0
lnRmax  = 5.5
zmin    = 0.0
zmax    = 0.664

table_ame = hdul_ame[1].data
table_rfae = hdul_rfae[1].data

print(table_ame.size)
print(table_rfae.size)

print(table_rfae.columns)
print(table_rfae["R_fae"])
print(table_rfae["R_ame"])

table_ame = table_ame[(table_ame["R_ame"] >=np.exp(lnRmin)) & (table_ame["R_ame"] <= np.exp(lnRmax)) & (table_ame["z"] >=zmin)
                      & (table_ame["z"] <=zmax)]

In [ ]:
area = 4109.3
cosmo = Nc.HICosmoDEXcdm()
cosmo.param_set_by_name("H0", 67.74)
cosmo.param_set_by_name("Omegab", 0.0486)
cosmo.param_set_by_name("Omegax", 0.6911)
cosmo.param_set_by_name("Omegac", 0.2603)
cosmo.param_set_by_name("w", -1.0)

cosmo.set_property("Omegac_fit", True)
cosmo.set_property("w_fit", False)




prim = Nc.HIPrimPowerLaw.new()
prim.param_set_by_name("ln10e10ASA", 3.02745)
prim.param_set_by_name("n_SA", 0.9667)
prim.set_property("ln10e10ASA_fit", True)


reion = Nc.HIReionCamb.new()

cosmo.add_submodel(prim)
cosmo.add_submodel(reion)

dist = Nc.Distance.new(2.0)
dist.prepare(cosmo)
print(cosmo.RH_Mpc() * dist.comoving(cosmo ,0.25))

tf = Nc.TransferFuncEH()

psml = Nc.PowspecMLTransfer.new(tf)
psml.require_kmin(1.0e-6)
psml.require_kmax(1.0e3)

psf = Ncm.PowspecFilter.new(psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()

mulf = Nc.MultiplicityFuncTinker.new()
mulf.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200.0)

hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
hmf.set_area(area * (np.pi / 180) ** 2)
hmf.prepare(cosmo)
hbias_Tinker = Nc.HaloBiasTinker.new(hmf)
cad = Nc.ClusterAbundance.new(hmf, None)
cad.set_area(area * (np.pi / 180) ** 2)
hmf.prepare(cosmo)

#σ_8 = 0.8159

In [ ]:
#mass proxy

cluster_m = Nc.ClusterMassLnrichExt(lnRichness_min = np.log(table_ame['R_ame'].min()) ,lnRichness_max =  np.log(table_ame['R_ame'].max()))
cluster_m.param_set_by_name("mu",4.164)
cluster_m.param_set_by_name("muM1", 0.5088)
cluster_m.param_set_by_name("muZ1", -4.494)
cluster_m.param_set_by_name("muM2", 0.1309)
cluster_m.param_set_by_name("muZ2", -9.685)
cluster_m.param_set_by_name("muMZ", -0.7108)

cluster_m.param_set_by_name("sigma0", 3.69)
cluster_m.param_set_by_name("sigmaM1", 1.048)
cluster_m.param_set_by_name("sigmaZ1", -1.562)
cluster_m.param_set_by_name("sigmaM2", 0.122)
cluster_m.param_set_by_name("sigmaZ2", -8.488)
cluster_m.param_set_by_name("sigmaMZ", 0.4834)

cluster_m.param_set_by_name("A0", 0.2235)
cluster_m.param_set_by_name("cut", 7.436)
cluster_m.param_set_by_name("cutM1", -0.5809)
cluster_m.param_set_by_name("cutZ1", 0.398)


cluster_m_ascaso = Nc.ClusterMassAscaso(lnRichness_min = np.log(table_ame['R_ame'].min()) ,lnRichness_max = np.log(table_ame['R_ame'].max()))
cluster_m_ascaso.param_set_by_name("mup0",4.368)
cluster_m_ascaso.param_set_by_name("mup1", 0.2566)
cluster_m_ascaso.param_set_by_name("mup2", 0.4474 )
cluster_m_ascaso.param_set_by_name("sigmap0",  0.3034 )
cluster_m_ascaso.param_set_by_name("sigmap1", -0.04202)
cluster_m_ascaso.param_set_by_name("sigmap2", 0.005575)



#photo-z
cluster_z = Nc.ClusterPhotozGauss (pz_min = zmin , pz_max = zmax)
#cluster_z = Nc.ClusterRedshiftNodist (z_min = zmin, z_max = zmax)

print("\n")
print(cluster_m.p(cosmo , 34.45, 0.1 , [np.log(35.46546)] , [1.175]))
print(cluster_m.intp(cosmo , 34 , 0.1))
print(cluster_m.intp_bin(cosmo , 34.45 , 0.1 , [1] , [10] , [2.90969]))
print("\n")
'''
print(cluster_z.p(cosmo , 32 , 0.66 , [0.657513] , [0,0.007513]))
print(cluster_z.intp(cosmo , 32 , 0.9))
print(cluster_z.intp_bin(cosmo , 32 , 0.9, [0.8] , [1.0] , [0,0.0513]))
print("\n")

'''

In [ ]:
masses  = np.linspace(np.log(10) *12.0, np.log(10) * 15.0,10000)
masses_10  = np.linspace(12.0 ,15,10000)
z  = np.linspace(0 ,0.664,10000)
relation = []
relation_z = []

for i in range(len(masses)):
    relation.append(cluster_m.p(cosmo , masses[i], 0.1 , [np.log(35.46546)] , None))
    relation_z.append(cluster_m.p(cosmo , 30, z[i] , [np.log(35.46546)] , None))
plt.plot(masses_10 , relation)
plt.ylabel(r'$P(ln\lambda|M,z)$')
plt.xlabel(r'$\log_{10} M$')
plt.title(r"$ln\lambda = %i $ " % (35))
plt.savefig("mass-richness-relation-35.jpg")
plt.show()

plt.plot(z , relation_z)
plt.ylabel(r'$P(ln\lambda|M,z)$')
plt.xlabel(r'$z$')
plt.show()

masses  = np.linspace(np.log(10) *14.0, np.log(10) * 16  ,10000)
masses_10  = np.linspace(14.0 ,16    ,10000)
relation = []
relation_z = []

for i in range(len(masses)):
    relation.append(cluster_m.p(cosmo , masses[i], 0.1 , [np.log(338.48)] , None))
    relation_z.append(cluster_m.p(cosmo , 30, z[i] , [np.log(338.48)] , None))
plt.plot(masses_10 , relation)
plt.ylabel(r'$P(ln\lambda|M,z)$')
plt.xlabel(r'$\log_{10} M$')
plt.title(r"$ln\lambda = %i$" % (338))
plt.savefig("mass-richness-relation-338.jpg")
plt.show()

plt.plot(z , relation_z)
plt.ylabel(r'$P(ln\lambda|M,z)$')
plt.xlabel(r'$z$')
plt.show()

In [ ]:
mset = Ncm.MSet.new_array([cosmo, cluster_m, cluster_z])
ncdata = Nc.DataClusterNCount.new(cad, "NcClusterPhotozGauss", "NcClusterMassLnrichExt")
#ncdata = Nc.DataClusterNCount.new(cad, "NcClusterPhotozGauss", "NcClusterMassAscaso")
ncdata.set_init(True)



richness = np.array(table_ame['R_ame'])
redshift = np.array(table_ame['z'])
redshift_err = np.array(table_ame['z_err'])

z_obs_params = Ncm.Matrix.new(len(redshift_err), 2)
for i in range(len(redshift_err)):
    for j in range(0, 2):
        if j == 0:
            z_obs_params.set(i, j, 0)
        if j == 1:
            z_obs_params.set(i, j,  redshift_err[i])
print(richness) 

ncdata.set_lnM_obs(Ncm.Matrix.new_array(richness, 1))
ncdata.set_z_obs(Ncm.Matrix.new_array(redshift, 1))
ncdata.set_z_obs_params(z_obs_params)
ncdata.true_data(False)

dset = Ncm.Dataset.new_array([ncdata])
likelihood = Ncm.Likelihood.new(dset)

experiment = Ncm.ObjDictStr()
experiment.set("likelihood", likelihood)
experiment.set("model-set", mset)
ser = Ncm.Serialize.new(Ncm.SerializeOpt.CLEAN_DUP)
ser.dict_str_to_yaml_file(experiment, "unbinned.yaml")

ca =[]
lnM_obs = ncdata.get_lnM_obs()
z_obs = ncdata.get_z_obs()
z_obs_params = ncdata.get_z_obs_params()

mulf = Nc.MultiplicityFuncDespali.new()
mulf.set_mdef(Nc.MultiplicityFuncMassDef.VIRIAL)


hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
hmf.set_area(area * (np.pi / 180) ** 2)

hmf.prepare(cosmo)
cad = Nc.ClusterAbundance.new(hmf, None)
hmf.prepare(cosmo)
cad.prepare(cosmo , cluster_z , cluster_m)
cad.set_area(area * (np.pi / 180) ** 2)
print(cad.n(cosmo , cluster_z , cluster_m))

#Fit
fit = Ncm.Fit.factory( Ncm.FitType.NLOPT, "ln-neldermead", likelihood, mset, Ncm.FitGradType.NUMDIFF_CENTRAL )#
fit.log_info()
#fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)
#fit.log_info()



In [ ]:
clusters_dt = Table([table_ame['R_ame'], table_ame['z'] , table_ame['z_err']], names=('richness' , 'z' , 'z_err'))

z = clusters_dt["z"]  # redshift
lnR = np.log(clusters_dt["richness"])  # ln(richness)

#bins:

z_bins = 1
d = (max(z) - min (z))/z_bins

r_bins = 100
d_r = (max(lnR) - min (lnR))/r_bins


print(f'Number of z bins: {z_bins}.\nNumber of lnR bins: {r_bins}.\nTotal bins: {z_bins * r_bins}.')

# redshift bins
z_0 = 0.0
z_1 = d
halos_bin_z =[]

for i in range(z_bins):
    cut_z = np.logical_and (clusters_dt['z'] > z_0, clusters_dt['z'] < z_1)
    halos_bin_z.append(clusters_dt[cut_z])
    z_0 = z_0 + d
    z_1 = z_1 + d
# mass bins
label = []
halos_bin_rz =[]
for i in range(z_bins):
    
    lnR_0 = min(lnR)
    lnR_1 = min(lnR) + d_r
    for j in range(r_bins):
        
        cut = np.logical_and (np.log(halos_bin_z[i]["richness"]) > lnR_0, np.log(halos_bin_z[i]["richness"]) < lnR_1)
        halos_bin_rz.append(halos_bin_z[i][cut])
        
        lnR_0 = lnR_0 + d_r
        lnR_1 = lnR_1 + d_r
        
print(f'Total bins in halos_bin_rz: {len(halos_bin_rz)}')

z_binned, z_err_binned, lnR_binned =[], [], [] 

for i in range(len(halos_bin_rz)):
    
    halos = halos_bin_rz[i]
    
    z_err_binned.append(halos["z_err"])
    z_binned.append(halos["z"])
    lnR_binned.append(np.log(halos["richness"]))   

lnR_mean, z_mean = [np.mean(l) for l in lnR_binned if len(l) > 1], [np.mean(k) for k in z_binned if len(k) > 1]
lnR_std, z_std = [np.std(l) for l in lnR_binned if len(l) > 1], [np.mean(k) for k in z_err_binned if len(k) > 1]


halos_mean = Table([np.exp(np.array(lnR_mean)), z_mean],
           names=('richness', 'z'))

halos_std = Table([np.exp(np.array(lnR_std)), z_std],
           names=('richness', 'z'))

print(f'Total bins in halos_mean: {len(halos_mean)}')

In [ ]:
cluster_z_no = Nc.ClusterRedshiftNodist (z_min = zmin, z_max = zmax)
richness = np.linspace(2.6, 5.5, len(halos_mean))
richness_plot = []
n_bin = []
n_bin_no = []
for i in range(len(richness)-1):
    
    n_bin.append(cad.intp_bin_d2n(cosmo , cluster_z , cluster_m , [richness[i]] ,[richness[i+1]],None ,[zmin] , [zmax] , [0 , z_std[i]]))
    n_bin_no.append(cad.intp_bin_d2n(cosmo , cluster_z , cluster_m_ascaso , [richness[i]] ,[richness[i+1]],None ,[zmin] , [zmax] , [0 , z_std[i]])/np.log(10))
    
    richness_plot.append((richness[i] + richness[i+1])/2)

print("%.5f , %.5f" % (zmin , zmax))
legend = ["quadratic" , "linear"]
    
    
plt.plot(richness_plot , n_bin , label = "%s" %(legend[0]))
#plt.plot(richness_plot , n_bin_no , label = "%s" %(legend[1]))

counts_rich, bins = np.histogram(np.log(table_ame['R_ame']), bins=(len(richness)-1))
plt.xlim(2.6 , 5.5)
plt.legend()
plt.xlabel(r"$ln(\lambda)$")
plt.ylabel(r"$n_M$")
plt.yscale("log")
plt.hist(np.log(table_ame['R_ame']), bins=(len(richness)-1))
plt.show()


diff = []
for i in range(len(n_bin)):
    diff.append((counts_rich[i] - n_bin[i])/counts_rich[i])
    

plt.scatter(richness_plot, diff, c='#a0d995', alpha=0.6774)
plt.legend()
plt.xlabel(r'$\ln\lambda$', size=14)
plt.ylabel('Erro Relativo std fit', size=14)
plt.axhline(y=0.0, color='black', linestyle='dotted')

print(diff)

## Cluster counts z distribution

In [ ]:
clusters_dt = Table([table_ame['R_ame'], table_ame['z'] , table_ame['z_err']], names=('richness' , 'z' , 'z_err'))

z = clusters_dt["z"]  # redshift
lnR = np.log(clusters_dt["richness"])  # ln(richness)

#bins:

z_bins = 100
d = (max(z) - min (z))/z_bins

r_bins = 1
d_r = (max(lnR) - min (lnR))/r_bins


print(f'Number of z bins: {z_bins}.\nNumber of lnR bins: {r_bins}.\nTotal bins: {z_bins * r_bins}.')

# redshift bins
z_0 = 0.0
z_1 = d
halos_bin_z =[]

for i in range(z_bins):
    cut_z = np.logical_and (clusters_dt['z'] > z_0, clusters_dt['z'] < z_1)
    halos_bin_z.append(clusters_dt[cut_z])
    z_0 = z_0 + d
    z_1 = z_1 + d
# mass bins
label = []
halos_bin_rz =[]
for i in range(z_bins):
    
    lnR_0 = min(lnR)
    lnR_1 = min(lnR) + d_r
    for j in range(r_bins):
        
        cut = np.logical_and (np.log(halos_bin_z[i]["richness"]) > lnR_0, np.log(halos_bin_z[i]["richness"]) < lnR_1)
        halos_bin_rz.append(halos_bin_z[i][cut])
        
        lnR_0 = lnR_0 + d_r
        lnR_1 = lnR_1 + d_r
        
print(f'Total bins in halos_bin_rz: {len(halos_bin_rz)}')

z_binned, z_err_binned, lnR_binned =[], [], [] 

for i in range(len(halos_bin_rz)):
    
    halos = halos_bin_rz[i]
    
    z_err_binned.append(halos["z_err"])
    z_binned.append(halos["z"])
    lnR_binned.append(np.log(halos["richness"]))   

lnR_mean, z_mean = [np.mean(l) for l in lnR_binned if len(l) > 1], [np.mean(k) for k in z_binned if len(k) > 1]
lnR_std, z_std = [np.std(l) for l in lnR_binned if len(l) > 1], [np.mean(k) for k in z_err_binned if len(k) > 1]


halos_mean = Table([np.exp(np.array(lnR_mean)), z_mean],
           names=('richness', 'z'))

halos_std = Table([np.exp(np.array(lnR_std)), z_std],
           names=('richness', 'z'))

print(f'Total bins in halos_mean: {len(halos_mean)}')

In [ ]:
z = np.linspace(zmin, zmax, len(halos_mean))

cluster_z_no = Nc.ClusterRedshiftNodist (z_min = zmin, z_max = zmax)
z_plot = []
n_bin = []
n_bin_no = []
for i in range(len(z)-1):
    n_bin.append(cad.intp_bin_d2n(cosmo , cluster_z , cluster_m , [lnRmin] ,[lnRmax],None ,[z[i]] , [z[i+1]] , [0.0,z_std[i]])*info['data']['recovery'][i]/info2['data']['recovery'][i]/np.log(10))
    n_bin_no.append(cad.intp_bin_d2n(cosmo , cluster_z , cluster_m, [lnRmin] ,[lnRmax],None ,[z[i]] , [z[i+1]] , [0.0,z_std[i]])/np.log(10))
    z_plot.append((z[i] + z[i+1])/2)

print("%.5f , %.5f" % (zmin , zmax))
legend = ["Completeness" , "Theory only"]
    
    
plt.plot(z_plot , n_bin , label = "%s" %(legend[0]))
plt.plot(z_plot , n_bin_no , label = "%s" %(legend[1]))

counts_z, bins = np.histogram(table_ame['z'], bins=(len(z)-1))
plt.xlim(table_ame['z'].min() , table_ame['z'].max())
plt.legend()
plt.xlabel(r"$z$")
plt.ylabel(r"$n_M$")
plt.yscale("log")
plt.hist(table_ame['z'], bins=(len(z)-1))
plt.savefig("detections modeling")
plt.show()


diff = []
for i in range(len(n_bin)):
    diff.append((counts_z[i] - n_bin[i])/counts_z[i])
    

plt.scatter(z_plot, diff, c='r', alpha=0.8)
plt.legend()
plt.xlabel(r'$z$', size=14)
plt.ylabel('Erro Relativo std fit', size=14)
plt.axhline(y=0.0, color='black', linestyle='dotted')

print(diff)

In [ ]:
from clevar.match_metrics import recovery
hdul_halos = fits.open("../halos/halos.fits")
hdul_ame = fits.open("richness_mcut_SNR 12_ame.fits")

logMmin = 13
logMmax = 15

table_halos = hdul_halos[1].data
table_ame = hdul_ame[1].data
table_halos = table_halos[(table_halos['upid'] == -1)  &  (table_halos['Mvir'] >= 10**logMmin) & (table_halos['Mvir'] <= 10**logMmax)]
table_halos['Mvir'] = table_halos['Mvir']/0.6774

print(table_halos.size)
print(table_ame.size)

print(table_ame.columns)

catalog_halos = clevar.ClCatalog('Halo catalog',ra=table_halos['RA'] ,dec=table_halos['DEC'] 
                                ,z=table_halos['redshift_R'], mass=table_halos['Mvir'])

catalog_ame = clevar.ClCatalog('Detection catalog', ra=table_ame['RA'], dec=table_ame['DEC']
                             , z=table_ame['z'], mass=table_ame['R_ame']  , mass_err=table_ame['R_ame_err'] ,z_err=table_ame['z_err'] , )
mt = clevar.match.ProximityMatch()
mt.load_matches(catalog_halos, catalog_ame, out_dir='temp')

halos_match, detections_match = clevar.match.get_matched_pairs(catalog_halos, catalog_ame, 'cross')

detections_match

zbins = np.linspace(0, 0.664, 93)
mbins = np.logspace(13, 15, 2)
rbins = np.logspace(0 , 5.8, 2)

info2 = recovery.plot(catalog_ame, 'cross', zbins, rbins, shape='steps')
plt.show()
info = recovery.plot(catalog_halos, 'cross', zbins, mbins, shape='steps')

In [ ]:
import selection
zbins = np.linspace(0.0, 0.664, 12)
mbins = np.logspace(13.0, 15, 12)

selection_ame = selection.NcClusterSelectionFunction()
info = recovery.plot(cat=catalog_halos, matching_type='cross', redshift_bins=zbins, mass_bins=mbins, shape='steps')


z   = Ncm.Vector.new(len(zbins)-1)
lnM = Ncm.Vector.new(len(mbins)-1)

completeness = Ncm.Matrix.new(len(zbins)-1 , len(mbins)-1)

for i in range(z.len()):
    z.set(i , (zbins[i]+zbins[i+1])/2)

for i in range(lnM.len()):
    lnM.set(i , (np.log(mbins[i])+np.log(mbins[i+1]))/2)


    
for i in range(z.len()):
    for j in range(lnM.len()):
        completeness.set(i , j , info['data']['recovery'][j][i])

selection_ame.nc_cluster_selection_function_completeness(z , lnM , completeness)

In [ ]:
zbins = np.linspace(0.0, 0.664, 7)
rbins = np.logspace(1.1 , 2.3, 7)

info2 = recovery.plot(catalog_ame, 'cross', zbins, rbins, shape='steps')
z = Ncm.Vector.new(len(zbins)-1)
lnR = Ncm.Vector.new(len(rbins)-1)
purity = Ncm.Matrix.new(len(zbins)-1 , len(rbins)-1)
for i in range(z.len()):
    z.set(i , (zbins[i]+zbins[i+1])/2)

for i in range(lnR.len()):
    lnR.set(i , (np.log(rbins[i])+np.log(rbins[i+1]))/2)

    
for i in range(z.len()):
    for j in range(lnR.len()):
        purity.set(i , j , info['data']['recovery'][j][i])
selection_ame.nc_cluster_selection_function_purity(z , lnR , purity)


In [ ]:
zbins = np.linspace(0.0, 0.664, 2)
mbins = np.logspace(13.0, 15, 100)

#info = recovery.plot(cat=catalog_halos, matching_type='cross', redshift_bins=zbins, mass_bins=mbins, shape='steps')
m_plot = []
for i in range(len(mbins)-1):
    m_plot.append((np.log10(mbins[i])+np.log10(mbins[i+1]))/2)
for j in range(len(zbins)-1):
    
    plt.step(m_plot , info['data']['recovery'][j] , label='[%.2f - %.2f]' % (zbins[j],zbins[j+1]))
    
plt.legend(loc=4)
plt.xlabel(r"$\log M$")
plt.ylabel("recovery rate")
plt.grid()